In [4]:
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')


rs = 132
dataset = pd.read_csv('./data/dataset.csv')
dataset.set_index('股票简称', inplace=True)

features = [
    '净资产收益率(%)', '资产报酬率(%)', 'EBITDA率(%)', '营业利润率(%)', '投入资本回报率(%)', 
    '资产负债率(%)', '权益乘数(%)', '速动比率(%)', '现金流动负债比率(%)', '长期资本负债率(%)',
    '营业收入增长率(%)', '资本保值增值率(%)', '总资产增长率(%)', '资本积累率(%)', '营业利润增长率(%)',
    '总资产周转率', '应收账款周转率', '流动资产周转率', '存货周转率', '现金资产比率(%)', 
    '数字技术应用', '商业模式变革', '智能制造', '现代信息系统',
    '客户集中度(%)', '供应商集中度(%)', '成本费用利润率(%)',
    '研发人员占比(%)', '研发营收比(%)', '发明专利申请数',
    '两权分离率(%)', '独董比例(%)', '董事会规模','股权集中度(%)',
    '员工人均营收比(%)', '提供岗位增长率(%)', '员工收入增长率(%)',
]
label_name = '因子得分'
unit_map = dataset[['股票代码', '行业代码', '所属省份']].to_dict()
# 获取数据集和标签值
y : pd.Series = dataset[label_name]
X : pd.DataFrame = dataset[features].copy(deep=True).astype("float")
# 数据预处理：1.极差标准化；2.数据集划分。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

{'股票代码': {'富奥股份': 30, '德赛电池': 49, '美的集团': 333, '潍柴动力': 338, '许继电气': 400, '冀东水泥': 401, '徐工机械': 425, '长虹美菱': 521, '柳工': 528, '顺钠股份': 533, '云南白药': 538, '佛山照明': 541, '威孚高科': 581, '古井贡酒': 596, '长安汽车': 625, '山推股份': 680, '恒逸石化': 703, '一汽解放': 800, '新希望': 876, '航天科技': 901, '钱江摩托': 913, '海信家电': 921, '盈峰环境': 967, '安泰科技': 969, '精工科技': 2006, '大族激光': 2008, '天奇股份': 2009, '山东威达': 2026, '思源电气': 2028, '华帝股份': 2035, '兔宝宝': 2043, '横店东磁': 2056, '中钢天源': 2057, '景兴纸业': 2067, '海鸥住工': 2084, '金智科技': 2090, '易普力': 2096, '浔兴股份': 2098, '广东鸿图': 2101, '三钢闽光': 2110, '威海广泰': 2111, '东南网架': 2135, '实益达': 2137, '拓邦股份': 2139, '通润装备': 2150, '智光电气': 2169, '芭田股份': 2170, '海得控制': 2184, '劲嘉股份': 2191, '大连重工': 2204, '鱼跃医疗': 2223, '江南化工': 2226, '奥特迅': 2227, '合兴包装': 2228, '鸿博股份': 2229, '大洋电机': 2249, '恩华药业': 2262, '华明装备': 2270, '东方雨虹': 2271, '川润股份': 2272, '万马股份': 2276, '天润工业': 2283, '亚太股份': 2284, '奇正藏药': 2287, '罗莱生活': 2293, '辉煌科技': 2296, '齐心集团': 2301, '美盈森': 2303, '普利特': 2324, '新朋股份': 2328, '英威腾': 2334, '科华数据': 2335, '格林美': 2340, '北京科锐